# Folium Project

## Description

### Installation Packages

In [2]:
%pip install pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pyodbc


   ---------------------------------------- 0.0/68.8 kB ? eta -:--:--
   ---------------------------------------- 68.8/68.8 kB 3.7 MB/s eta 0:00:00


### Code

In [18]:
import pyodbc

# Définissez les informations de connexion
# Aller dans SSMIS -> clique droit sur AdventureWorks2022 -> Properties et récupérer le nom du server et de la database
server = 'LAPTOP-SQ0G8B4V\SQLEXPRESS'
database = 'AdventureWorks2022'

# Établissez la connexion
conn = pyodbc.connect(
    f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Windows Authentication"
)


In [16]:
# Exécutez des requêtes SQL et traitez les résultats avec pandas
query = "SELECT * FROM Person.Person"
data = pd.read_sql(query, conn)


C:\Users\veyss\AppData\Local\Temp\ipykernel_48116\1317191718.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [17]:
data

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19967,20773,IN,False,None,Crystal,None,Guo,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D0A499A2-C258-43E9-AB06-F61543A71134,2014-03-19
19968,20774,IN,False,None,Isabella,F,Richardson,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A4F7F248-82F3-43D5-B5A3-8BD1CB610254,2013-07-29
19969,20775,IN,False,None,Crystal,S,He,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CD0B82D0-3409-4497-A65F-A68E75EE26CB,2014-03-12
19970,20776,IN,False,None,Crystal,None,Zheng,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",386E35E8-97B3-43E4-9B5F-9594C2FF5CEA,2014-01-14


In [ ]:
# Fermez la connexion
conn.close()